In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch
import cv2

Matplotlib is building the font cache; this may take a moment.
/data/chengjingyuan/miniconda3/envs/tracking_SAM/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import groundingdino.datasets.transforms as T
from groundingdino.models import build_model
from groundingdino.util import box_ops
from groundingdino.util.slconfig import SLConfig
from groundingdino.util.utils import clean_state_dict
from groundingdino.util.inference import predict

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
/data/chengjingyuan/miniconda3/envs/tracking_SAM/lib/python3.8/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [3]:
config_file_path = './tracking_SAM/third_party/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py'
model_path = './pretrained_weights/groundingdino_swint_ogc.pth'

args = SLConfig.fromfile(config_file_path) 
device = 'cpu'

dino_model = build_model(args)

checkpoint = torch.load(model_path, map_location='cpu')
log = dino_model.load_state_dict(clean_state_dict(checkpoint['model']), strict=False)
dino_model.eval()
dino_model = dino_model.to(device)

print(log)

/data/chengjingyuan/miniconda3/envs/tracking_SAM/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525541702/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


_IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])


In [ ]:
test_img_path = "./sample_data/DAVIS_bear/images/00000.jpg"
image_np = np.asarray(Image.open(test_img_path).convert("RGB"))

transform = T.Compose(
    [
        T.RandomResize([800], max_size=1333),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

img_chw, _ = transform(Image.fromarray(image_np), None)

In [ ]:
text_prompt = 'bear'

BOX_TRESHOLD = 0.3
TEXT_TRESHOLD = 0.25

In [ ]:
boxes, logits, phrases = predict(
    model=dino_model, 
    image=img_chw, 
    caption=text_prompt, 
    box_threshold=BOX_TRESHOLD, 
    text_threshold=TEXT_TRESHOLD,
    device=device
)

In [ ]:
H, W, _ = image_np.shape

boxes_xyxy = box_ops.box_cxcywh_to_xyxy(boxes) * torch.Tensor([W, H, W, H])

In [ ]:
viz_img = image_np.copy()

for box, phrase in zip(boxes_xyxy, phrases):
    box = box.cpu().numpy().astype(np.int32)
    viz_img = cv2.rectangle(viz_img, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
    viz_img = cv2.putText(viz_img, phrase, (box[0], box[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

plt.imshow(viz_img)

In [ ]:
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = "./pretrained_weights/sam_vit_h_4b8939.pth"  # default model

model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

In [ ]:
predictor.set_image(image_np)

In [ ]:
assert len(boxes_xyxy) == 1

In [ ]:
input_box = boxes_xyxy[0].cpu().numpy()

In [ ]:
masks, _, _ = predictor.predict(
    point_coords=None,
    point_labels=None,
    box=input_box[None, :],
    multimask_output=False,
)

In [ ]:
plt.imshow(masks[0])